**Implementacija predikcije ocene parfema uz koriscenje ANN**

Pre svega treba importovati sve potrebne pakete koji ce nam sluziti za analizu.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
import re
from ast import literal_eval
from tensorflow.keras.preprocessing.sequence import pad_sequences


Funckija *parse_season_ratings* sluzi za za parsiranje polja godisnja doba iz skupa podataka, dok funckija *consolidate_notes* objedinjuje/konsoliduje base, middle i top note u jednu kolekciju

In [2]:
def parse_season_ratings(rating_str):
    pattern = r'([A-Za-z]+):\s*([0-9.]+)%'
    return {season: float(percent) for season, percent in re.findall(pattern, rating_str)}

def consolidate_notes(notes):
    all_notes = []
    for note_type in ['Top Notes', 'Middle Notes', 'Base Notes']:
        if note_type in notes:
            all_notes.extend(notes[note_type])
    return all_notes


Ucitavanje skupa podataka

In [3]:
file_path = "../datasets/mainDataset.csv"
data = pd.read_csv(file_path, delimiter='|')


Izdvajanje relevantnih polja iz skupa podataka koji ce biti ulaz u algoritme.
Bilo su isprobane razne kombinacije.. (popricati sa vukasinom sta cemo da napisemo)

In [4]:
data['Accords'] = data['Accords'].apply(literal_eval)
data['Notes'] = data['Notes'].apply(literal_eval)
data['Votes'] = data['Rating'].apply(lambda x: literal_eval(x)['votes'])
data['Rating'] = data['Rating'].apply(lambda x: literal_eval(x)['rating'])
data['Season ratings'] = data['Season ratings'].apply(parse_season_ratings)
data['Day ratings'] = data['Day ratings'].apply(parse_season_ratings)
data['Designers'] = data['Designers'].apply(literal_eval)


Stvaranje recorda i formiranje Data Frame koji sadrzi svaki parfem iz skupa podataka u pogodnom obliku i svih nota JEDNOG parfema

In [5]:
records = []
for _, row in data.iterrows():
    record = {
        "Brand": row["Brand"],
        "Gender": row["Gender"],
        "Longevity": row["Longevity"],
        "Sillage": row["Sillage"],
        "Rating": row["Rating"],
        "Votes": row["Votes"],
        "Season_Winter": row["Season ratings"].get("Winter", 0),
        "Season_Spring": row["Season ratings"].get("Spring", 0),
        "Season_Summer": row["Season ratings"].get("Summer", 0),
        "Season_Fall": row["Season ratings"].get("Fall", 0),
        "Day": row["Day ratings"].get("Day", 0),
        "Night": row["Day ratings"].get("Night", 0)
    }
    records.append(record)

structured_df = pd.DataFrame(records)
structured_df['All Notes'] = data['Notes'].apply(consolidate_notes) 


Note su tekstualne vrednosti koje mreža ne može direktno da koristi, pa se svaka nota pretvara u jedinstveni broj (ID). Problem je što parfemi imaju različit broj nota, a mreži trebaju ulazi iste dužine. Zato se sve liste nota skraćuju ili dopunjavaju nulama do iste dužine (20). Tako svaki parfem dobija numerički niz fiksne veličine i mreža može da uči iz tih podataka.
Odabran je broj 20 jer predtsvalja optimalan broj nota po parfemu, sa manjim brojevima (npr. 10) su dobijani losiji rezultati.

In [6]:

all_unique_notes = list(set(note for notes in structured_df['All Notes'] for note in notes)) 
note_to_id = {note: idx for idx, note in enumerate(all_unique_notes)} 
structured_df['Note_IDs'] = structured_df['All Notes'].apply(lambda notes: [note_to_id[note] for note in notes if note in note_to_id]) 
max_len = 20 
structured_df['Note_IDs_Padded'] = pad_sequences(structured_df['Note_IDs'], maxlen=max_len, padding='post').tolist() 



Endokiranje, odnsno pretvaranje kategoricnih vrednosti u numericke.

In [7]:
label_encoder = LabelEncoder()
structured_df['Gender'] = label_encoder.fit_transform(structured_df['Gender'])
structured_df['Brand'] = label_encoder.fit_transform(structured_df['Brand'])


Iz podataka se izdvajaju ulazi i izlazi za mrežu. Kao cilj (y) koriste se ocena parfema, a ulazi (X) su sve ostale karakteristike osim kolona koje nisu pogodne za direktno treniranje, kao što su originalne liste nota. Posebno se čuvaju i obeležja vezana za note u obliku niza fiksne dužine (X_notes).
Numeričke kolone poput trajnosti, sillage-a, broja glasova i sezonskih ocena normalizuju se pomoću StandardScaler tako da imaju prosečnu vrednost nula i standardnu devijaciju jedan, što mreži olakšava učenje i dalo je bolje rezulate u odnsosu kad nismo koristili.
Na kraju se ceo skup deli na trening i test deo u odnosu 80:20, i to odvojeno za obične atribute (X) i za note (X_notes), da bi se model mogao trenirati i kasnije testirati na podacima koje nije video.

Note, s druge strane, predstavljaju sekvencu ID-jeva, gde svaki ID označava jednu notu parfema. Broj nota nije isti za svaki parfem, pa su prethodno skraćene ili dopunjene nulama do fiksne dužine (Note_IDs_Padded). Takav oblik je pogodan za embedding sloj u mreži, gde mreža uči reprezentaciju svake note kao vektora. Dakle, X_notes ide u embedding sloj (ili sekvencijalni deo mreže), a X ide u standardni dense sloj, a kasnije se oba spoje i dalje obrađuju zajedno u mreži.

In [8]:
X = structured_df.drop(columns=['Rating', 'All Notes', 'Note_IDs', 'Note_IDs_Padded']) 
y = structured_df['Rating']
X_notes = np.array(structured_df['Note_IDs_Padded'].tolist())

numerical_features = ['Longevity','Sillage','Votes','Season_Winter','Season_Spring','Season_Summer','Season_Fall','Day','Night']
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_notes, X_test_notes = train_test_split(X_notes, test_size=0.2, random_state=42)


Pravi se neuronska mrežu sa dva ulaza. Jedan ulaz prima numeričke i enkodovane karakteristike parfema, dok drugi prima note u obliku niza ID-jeva koji se embedding-uju u vektore. Embedding sloj pretvara svaku notu u vektor dimenzije 32, a zatim se svi vektori nota spajaju u jedan dugačak vektor. Ovaj vektor se kombinuje sa ulazom za ostale karakteristike i prolazi kroz dva dense sloja sa ReLU aktivacijom, na kraju dajući jednu vrednost predikcije ocene parfema. Model se kompajlira sa Adam optimizatorom, gubitkom MSE i metrikom MAE, spreman za treniranje regresije.

ReLU je korišćen jer parfemski podaci imaju složene, nelinearne odnose između karakteristika i nota, pa mreži treba aktivacija koja može da ih modeluje. Adam je odabran jer omogućava brzo i stabilno treniranje na heterogenim ulazima, gde su zajedno numeričke karakteristike i embeddingovane note.

Learning rate 0.005 se pokazao kao najbolji u poredjenjeu sa ....  ***DOPUNITI SA PRIMEROM STA SMO PROBALI***

In [9]:

note_input = Input(shape=(max_len,))
note_embedding = Embedding(input_dim=len(all_unique_notes), output_dim=32)(note_input)
note_flattened = Flatten()(note_embedding)

structured_input = Input(shape=(X_train.shape[1],))
concatenated = Concatenate()([structured_input, note_flattened])

dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1)(dense_2)

ann_model = Model(inputs=[structured_input, note_input], outputs=output)
ann_model.compile(optimizer=Adam(learning_rate=0.005), loss='mse', metrics=['mae'])


Model se trenira 200 epoha sa veličinom batch-a 32. Parametar verbose=0 znači da tokom treniranja neće prikazivati detalje napretka, pa se proces odvija u pozadini bez ispisa.
200 epoha se prikazalo kao optimalan broj epoha kada uporedimo tacnost ali i brzinu izvrsavanaj naspram dobijene tacnosti. Sa manjim brojem epohda su dobijani losiiji rezultati, a sa vecim brojem epohda nismo dopirneli rezultatima. ***DOPUNITI SA PRIMEROM STA SMO PROBALI***

In [10]:
ann_model.fit([X_train, X_train_notes], y_train, epochs=200, batch_size=32, verbose=0)


In [11]:

ann_loss, ann_mae = ann_model.evaluate([X_test, X_test_notes], y_test, verbose=0)
ann_predictions = ann_model.predict([X_test, X_test_notes]).flatten()
ann_rmse = np.sqrt(mean_squared_error(y_test, ann_predictions))

print(f"ANN Mean Absolute Error (MAE): {ann_mae}")
print(f"ANN Root Mean Squared Error (RMSE): {ann_rmse}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
ANN Mean Absolute Error (MAE): 0.20729228854179382
ANN Root Mean Squared Error (RMSE): 0.26834067042874504
